In [1]:
import random
import folium
import numpy as np
import pandas as pd
import networkx as nx
from scipy.spatial import ConvexHull
from shapely.geometry import LineString, Point

# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
file_path = '/content/drive/Shareddrives/CHEC/data_chec/datos/Data_Total/REDMT.csv'
redmt = pd.read_csv(file_path)

# Especifica los tipos de datos deseados para cada columna
redmt_column_types = {
    'CODE': 'object',
    'PHASES': 'int64',
    'FPARENT': 'object',
    'ELNODE1': 'object',
    'ELNODE2': 'object',
    'CONDUCTOR': 'object',
    'NEUTRAL': 'object',
    'LENGTH': 'float64',
    'KVNOM': 'float64',
    'CLASS': 'int64',
    'TOWNER': 'object',
    'FECHA': 'datetime64[ns]',
    'MATERIALCONDUCTOR': 'object',
    'TIPOCONDUCTOR': 'object',
    'CALIBRECONDUCTOR': 'object',
    'GUARDACONDUCTOR': 'int64',
    'NEUTROCONDUCTOR': 'int64',
    'NEUTRO_SECUNDARIOCONDUCTOR': 'int64',
    'NIVEL1CONDUCTOR': 'int64',
    'NIVEL2CONDUCTOR': 'int64',
    'NIVEL3CONDUCTOR': 'int64',
    'NIVEL4CONDUCTOR': 'int64',
    'ACOMETIDACONDUCTOR': 'int64',
    'NEUTRO': 'object',
    'CALIBRENEUTRO': 'object',
    'FECHA_ACT': 'object',
    'CAPACITY': 'float64',
    'RESISTANCE': 'float64',
    'LONGITUD': 'float64',
    'LATITUD': 'float64',
    'LONGITUD2': 'float64',
    'LATITUD2': 'float64',
    'DEP': 'object',
    'MUN': 'object',
}

# Convierte las columnas al tipo especificado
redmt = redmt.astype(redmt_column_types)

file_path = '/content/drive/Shareddrives/CHEC/data_chec/datos/Data_Total/APOYOS.csv'
apoyos = pd.read_csv(file_path)

# Especifica los tipos de datos deseados para cada columna
apoyos_column_types = {
    'CODE': 'object',
    'ASSEMBLY': 'object',
    'TOWNER': 'object',
    'FECHA': 'datetime64[ns]',
    'TIPO': 'object',
    'CLASE': 'object',
    'MATERIAL': 'object',
    'LONG_APOYO': 'object',
    'TIERRA_PIE': 'object',
    'VIENTOS': 'object',
    'LONGITUD': 'float64',
    'LATITUD': 'float64',
    'DEP': 'object',
    'MUN': 'object',
}

# Convierte las columnas al tipo especificado
apoyos = apoyos.astype(apoyos_column_types)

file_path = '/content/drive/Shareddrives/CHEC/data_chec/datos/Data_Total/SWITCHES.csv'
switches = pd.read_csv(file_path)

# Especifica los tipos de datos deseados para cada columna
switches_column_types = {
    'CODE': 'object',
    'ASSEMBLY': 'object',
    'PHASES': 'int64',
    'FPARENT': 'object',
    'KV': 'float64',
    'STATE': 'object',
    'LINESECTIO': 'object',
    'FECHA': 'datetime64[ns]',
    'LONGITUD': 'float64',
    'LATITUD': 'float64',
    'DEP': 'object',
    'MUN': 'object',
}

# Convierte las columnas al tipo especificado
switches = switches.astype(switches_column_types)

file_path = '/content/drive/Shareddrives/CHEC/data_chec/datos/Data_Total/TRAFOS.csv'
trafos = pd.read_csv(file_path)

# Especifica los tipos de datos deseados para cada columna
trafos_column_types = {
    'CODE': 'object',
    'PHASES': 'int64',
    'FPARENT': 'object',
    'OWNER1': 'object',
    'TRFTYPE': 'object',
    'ELNODE': 'object',
    'IMPEDANCE': 'float64',
    'DATE_FAB': 'object',
    'MARCA': 'object',
    'TIPO_SUB': 'object',
    'FECHA': 'datetime64[ns]',
    'GRUPO015': 'int64',
    'FECHA_ACT': 'object',
    'KVA': 'float64',
    'KV1': 'float64',
    'LONGITUD': 'float64',
    'LATITUD': 'float64',
    'DEP': 'object',
    'MUN': 'object',
}

# Convierte las columnas al tipo especificado
trafos = trafos.astype(trafos_column_types)

In [3]:
redmt.to_pickle('/content/drive/Shareddrives/CHEC/data_chec/datos/Data_Total/REDMT.pkl')
trafos.to_pickle('/content/drive/Shareddrives/CHEC/data_chec/datos/Data_Total/TRAFOS.pkl')
switches.to_pickle('/content/drive/Shareddrives/CHEC/data_chec/datos/Data_Total/SWITCHES.pkl')
apoyos.to_pickle('/content/drive/Shareddrives/CHEC/data_chec/datos/Data_Total/APOYOS.pkl')

In [3]:
año = 2023
mes = 1
mun = 'AGUADAS'

redmt_seleccionado = redmt.loc[(redmt['FECHA'].dt.year == año) & (redmt['FECHA'].dt.month == mes) & (redmt['MUN'] == mun)]
trafos_seleccionado = trafos.loc[(trafos['FECHA'].dt.year == año) & (trafos['FECHA'].dt.month == mes) & (trafos['MUN'] == mun)]
switches_seleccionado = switches.loc[(switches['FECHA'].dt.year == año) & (switches['FECHA'].dt.month == mes) & (switches['MUN'] == mun)]
apoyos_seleccionado = apoyos.loc[(apoyos['FECHA'].dt.year == año) & (apoyos['FECHA'].dt.month == mes) & (apoyos['MUN'] == mun)]

In [4]:
def filtrar_aguas_abajo(grafo, nodo_inicial):
    aguas_abajo = set(nx.descendants(grafo, nodo_inicial))
    aguas_abajo.add(nodo_inicial)
    return aguas_abajo

G = nx.DiGraph()
for _, row in redmt_seleccionado.iterrows():
    point1 = (row["LATITUD"], row["LONGITUD"])
    point2 = (row["LATITUD2"], row["LONGITUD2"])
    if row["ORDER_"] == 1:
        G.add_edge(point1, point2, id=row["CODE"])
    else:
        G.add_edge(point2, point1, id=row["CODE"])

# Seleccionar un nodo inicial válido del grafo
nodo_inicial = list(G.nodes)[335]  # Selecciona el primer nodo como ejemplo
print(f"Nodo inicial seleccionado: {nodo_inicial}")

# Filtrar nodos aguas abajo
nodos_aguas_abajo = filtrar_aguas_abajo(G, nodo_inicial)

# Detectar los puntos dispersos que están cerca de las conexiones aguas abajo
puntos_verdes_id = set()  # IDs de los puntos que cumplen el criterio
puntos_verdes_coord = set()
for _, row in redmt_seleccionado.iterrows():
    point1 = (row["LATITUD"], row["LONGITUD"])
    point2 = (row["LATITUD2"], row["LONGITUD2"])
    if point1 in nodos_aguas_abajo and point2 in nodos_aguas_abajo:
        line = LineString([point1, point2])
        for _, row in trafos_seleccionado.iterrows():
            p = Point(row["LATITUD"],row["LONGITUD"])
            if line.distance(p) < 0.001:  # Umbral de proximidad
                puntos_verdes_id.add(row["CODE"])
                puntos_verdes_coord.add((row["LATITUD"], row["LONGITUD"]))
        for _, row in apoyos_seleccionado.iterrows():
            p = Point(row["LATITUD"],row["LONGITUD"])
            if line.distance(p) < 0.001:  # Umbral de proximidad
                puntos_verdes_id.add(row["CODE"])
                puntos_verdes_coord.add((row["LATITUD"], row["LONGITUD"]))
        for _, row in switches_seleccionado.iterrows():
            p = Point(row["LATITUD"],row["LONGITUD"])
            if line.distance(p) < 0.001:  # Umbral de proximidad
                puntos_verdes_id.add(row["CODE"])
                puntos_verdes_coord.add((row["LATITUD"], row["LONGITUD"]))

# Convierte el set en un array de NumPy
puntos_array = np.array(list(puntos_verdes_coord))
# Calcula el convex hull
hull = ConvexHull(puntos_array)
# Obtiene los vértices del polígono en el orden del hull
polygon_puntos = puntos_array[hull.vertices].tolist()



# Crear el mapa
m = folium.Map(location=[redmt_seleccionado.LATITUD.mean(), redmt_seleccionado.LONGITUD.mean()], zoom_start=12)

# Dibuja el polígono en el mapa
folium.Polygon(locations=polygon_puntos, color='orange', fill=True, fill_opacity=0.3, weight=0).add_to(m)


# Añadir las conexiones
for _, row in redmt_seleccionado.iterrows():
    point1 = (row["LATITUD"], row["LONGITUD"])
    point2 = (row["LATITUD2"], row["LONGITUD2"])
    if (point1 in nodos_aguas_abajo and point2 in nodos_aguas_abajo):
      color = 'blue'
    else:
      color = 'gray'
    folium.PolyLine(
        [point1, point2],
        color=color,
        weight=3,
        opacity=0.8,
        popup=f"Connection ID: {row['CODE']}, Latitud: {row['LATITUD']}, Longitud: {row['LONGITUD']}, Latitud 2: {row['LATITUD2']}, Longitud 2: {row['LONGITUD2']}, ORDER: {row['ORDER_']}"
    ).add_to(m)


# Añadir los puntos
for _, row in trafos_seleccionado.iterrows():
    color = "green" if row["CODE"] in puntos_verdes_id else "gray"
    folium.CircleMarker(
        location=(row["LATITUD"], row["LONGITUD"]),
        radius=5,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.6,
        popup=f"Point ID: {row['CODE']}, Latitud: {row['LATITUD']}, Longitud: {row['LONGITUD']}"
    ).add_to(m)

# Añadir los puntos
for _, row in apoyos_seleccionado.iterrows():
    color = "green" if row["CODE"] in puntos_verdes_id else "gray"
    folium.CircleMarker(
        location=(row["LATITUD"], row["LONGITUD"]),
        radius=5,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.6,
        popup=f"Point ID: {row['CODE']}, Latitud: {row['LATITUD']}, Longitud: {row['LONGITUD']}"
    ).add_to(m)

# Añadir los puntos
for _, row in switches_seleccionado.iterrows():
    color = "green" if row["CODE"] in puntos_verdes_id else "gray"
    folium.CircleMarker(
        location=(row["LATITUD"], row["LONGITUD"]),
        radius=5,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.6,
        popup=f"Point ID: {row['CODE']}, Latitud: {row['LATITUD']}, Longitud: {row['LONGITUD']}"
    ).add_to(m)

Nodo inicial seleccionado: (5.66564447604595, -75.42546589956261)


/usr/local/lib/python3.10/dist-packages/shapely/measurement.py:72: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)
/usr/local/lib/python3.10/dist-packages/shapely/measurement.py:72: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)
/usr/local/lib/python3.10/dist-packages/shapely/measurement.py:72: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)


In [6]:
#index = list(G.nodes).index((5.684000883650678,-75.53113162409974))
#print(index)

In [30]:
m

Output hidden; open in https://colab.research.google.com to view.

In [ ]:


def generar_datos_aleatorios(num_puntos, num_ramales, lat_range, lon_range):
    puntos = [
        (random.uniform(*lat_range), random.uniform(*lon_range))
        for _ in range(num_puntos)
    ]
    ramales = []
    for i in range(num_ramales):
        p1, p2 = random.sample(puntos, 2)
        direction = random.choice([1, -1])
        if direction == 1:
            ramales.append((f"R{i}", *p1, *p2, direction))
        else:
            ramales.append((f"R{i}", *p2, *p1, direction))
    df = pd.DataFrame(ramales, columns=["id", "x1", "y1", "x2", "y2", "direction"])
    return df

def generar_puntos_distribuidos(num_puntos, lat_range, lon_range):
    puntos = [
        {
            "id": f"P{i}",
            "lat": random.uniform(*lat_range),
            "lon": random.uniform(*lon_range)
        }
        for i in range(num_puntos)
    ]
    return pd.DataFrame(puntos)

def filtrar_aguas_abajo(grafo, nodo_inicial):
    aguas_abajo = set(nx.descendants(grafo, nodo_inicial))
    aguas_abajo.add(nodo_inicial)
    return aguas_abajo

# Generar datos de nodos y ramales
df_aleatorio = generar_datos_aleatorios(
    num_puntos=50,
    num_ramales=25,
    lat_range=(19.0, 20.0),
    lon_range=(-100.0, -99.0)
)

puntos_distribuidos = generar_puntos_distribuidos(
    num_puntos=200,
    lat_range=(19.0, 20.0),
    lon_range=(-100.0, -99.0)
)

G = nx.DiGraph()
for _, row in df_aleatorio.iterrows():
    point1 = (row["x1"], row["y1"])
    point2 = (row["x2"], row["y2"])
    if row["direction"] == 1:
        G.add_edge(point1, point2, id=row["id"])
    else:
        G.add_edge(point2, point1, id=row["id"])

# Seleccionar un nodo inicial válido del grafo
nodo_inicial = list(G.nodes)[10]  # Selecciona el primer nodo como ejemplo
print(f"Nodo inicial seleccionado: {nodo_inicial}")

# Filtrar nodos aguas abajo
nodos_aguas_abajo = filtrar_aguas_abajo(G, nodo_inicial)

# Detectar los puntos dispersos que están cerca de las conexiones aguas abajo
puntos_verdes = set()  # IDs de los puntos que cumplen el criterio
for _, row in df_aleatorio.iterrows():
    point1 = (row["x1"], row["y1"])
    point2 = (row["x2"], row["y2"])
    if point1 in nodos_aguas_abajo and point2 in nodos_aguas_abajo:
        line = LineString([point1, point2])
        for _, punto in puntos_distribuidos.iterrows():
            p = Point(punto["lat"], punto["lon"])
            if line.distance(p) < 0.009:  # Umbral de proximidad
                puntos_verdes.add(punto["id"])

# Crear el mapa
m = folium.Map(location=[19.5, -99.5], zoom_start=12)

# Añadir las conexiones
for _, row in df_aleatorio.iterrows():
    point1 = (row["x1"], row["y1"])
    point2 = (row["x2"], row["y2"])
    color = "blue" if point1 in nodos_aguas_abajo and point2 in nodos_aguas_abajo else "gray"
    folium.PolyLine(
        [point1, point2],
        color=color,
        weight=3,
        opacity=0.8,
        popup=f"Connection ID: {row['id']}"
    ).add_to(m)

# Añadir los puntos dispersos
for _, punto in puntos_distribuidos.iterrows():
    color = "green" if punto["id"] in puntos_verdes else "gray"
    folium.CircleMarker(
        location=(punto["lat"], punto["lon"]),
        radius=5,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.6,
        popup=f"Point ID: {punto['id']}"
    ).add_to(m)

Nodo inicial seleccionado: (19.718330064892506, -99.85367634121526)


In [ ]:
m